## Импорты и Утилиты

In [ ]:
from unsloth import FastLanguageModel

import torch
import gc
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from tabulate import tabulate

# Для хранения результатов тестов
results = []

def clean_memory():
    """Принудительная очистка VRAM и Garbage Collector"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()
    print("🧹 Память очищена")

def print_memory():
    """Вывод занятой памяти"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        print(f"📊 VRAM: Занято {allocated:.2f} ГБ | Зарезервировано {reserved:.2f} ГБ")
    else:
        print("❌ GPU не найден")

print_memory()

## Математика памяти (Теория)

In [ ]:
# Функция для расчета потребления памяти
def estimate_vram(params_b, quant_type="fp16", context=8192):
    bytes_per_param = {
        "fp16": 2,      # 16 bit = 2 bytes
        "int8": 1,      # 8 bit = 1 byte
        "int4": 0.5     # 4 bit = 0.5 bytes (packed)
    }
    
    bpp = bytes_per_param.get(quant_type, 2)
    
    # Вес модели
    model_size = params_b * bpp
    
    # KV-Cache (упрощенная формула для демонстрации: 2 * layers * hidden * context...)
    # Грубая эвристика: ~0.5 МБ на 1к токенов для 7B моделей в FP16
    kv_cache = (context / 1000) * 0.5 * (bpp / 2) 
    
    total = model_size + kv_cache
    return f"Модель {params_b}B ({quant_type}): ~{total:.2f} ГБ VRAM (при контексте {context})"

print("--- Ваши лимиты: RTX 4070 (8 GB) ---")
print(estimate_vram(7, "fp16")) # Не влезет
print(estimate_vram(7, "int4")) # Влезет легко
print(estimate_vram(32, "int4")) # Не влезет (нужен A100 или RAM offload)

## Qwen3-4B в полном весе (BF16)

In [ ]:
clean_memory()

model_id = "Qwen/Qwen3-4B"
print(f"🏋️ Загрузка {model_id} в полном весе (BF16)...")

# Грузим в bfloat16 (2 байта на параметр)
# device_map="auto" спасет от краша, если 8ГБ чуть-чуть не хватит (выгрузит часть в RAM)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

print_memory()

# Тест генерации
input_text = "Explain the concept of entropy in simple terms."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

start = time.time()
output = model.generate(**inputs, max_new_tokens=100)
end = time.time()

tps = 100 / (end - start)
print(f"⏱ Скорость (BF16): {tps:.2f} токенов/сек")
results.append(["Qwen3-4B (BF16)", f"{torch.cuda.memory_allocated(0)/1024**3:.2f} GB", f"{tps:.2f} t/s"])

del model, tokenizer
clean_memory()

## Qwen3-4B квантованная (NF4)

In [ ]:
clean_memory()

print(f"🤏 Загрузка {model_id} в квантовании NF4...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

print_memory() # Должно быть около 3 ГБ

inputs = tokenizer("Explain the concept of entropy in simple terms.", return_tensors="pt").to("cuda")

start = time.time()
output = model.generate(**inputs, max_new_tokens=100)
end = time.time()

tps = 100 / (end - start)
print(f"⏱ Скорость (NF4): {tps:.2f} токенов/сек")
results.append(["Qwen3-4B (NF4)", f"{torch.cuda.memory_allocated(0)/1024**3:.2f} GB", f"{tps:.2f} t/s"])

del model, tokenizer
clean_memory()

## Qwen3-8B (Transformers + Thinking Mode)

In [ ]:
clean_memory()

model_id_8b = "Qwen/Qwen3-8B"
print(f"🧠 Загрузка {model_id_8b} (HF NF4) с Thinking Mode...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id_8b,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id_8b)

# Подготовка промпта для Thinking Mode
prompt = "How many 'r's are in the word 'Strawberry'?"
messages = [{"role": "user", "content": prompt}]

# Включаем enable_thinking=True
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True 
)
model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

start = time.time()
generated_ids = model.generate(**model_inputs, max_new_tokens=512)
end = time.time()

# Декодинг (разделяем мысли и ответ)
# В Qwen3 мысли обычно заключены в теги <think>...</think>
output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

print("\n--- RAW OUTPUT ---")
print(output_text[:500] + "...") # Печатаем начало, чтобы увидеть теги

tps = (len(generated_ids[0]) - len(model_inputs.input_ids[0])) / (end - start)
print(f"\n⏱ Скорость (HF 8B): {tps:.2f} токенов/сек")
results.append(["Qwen3-8B (HF NF4)", f"{torch.cuda.memory_allocated(0)/1024**3:.2f} GB", f"{tps:.2f} t/s"])

del model, tokenizer
clean_memory()

## Qwen3-8B через Unsloth (Сравнение скорости)

In [ ]:
clean_memory()

# Квантованная модель Unsloth
unsloth_model_id = "unsloth/Qwen3-8B-bnb-4bit" 

print(f"🐇 Unsloth: Загрузка {unsloth_model_id}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=unsloth_model_id,
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model) # Включаем магию оптимизации

inputs = tokenizer(["How many 'r's are in the word 'Strawberry'?"], return_tensors="pt").to("cuda")

# Прогрев
_ = model.generate(**inputs, max_new_tokens=1)

start = time.time()
outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
end = time.time()

tps = 512 / (end - start) # Примерный подсчет
print(f"⏱ Скорость (Unsloth 8B): {tps:.2f} токенов/сек")
results.append(["Qwen3-8B (Unsloth)", f"{torch.cuda.memory_allocated(0)/1024**3:.2f} GB", f"{tps:.2f} t/s"])

del model, tokenizer
clean_memory()

## Llama.cpp (GGUF & Hybrid Inference)

In [ ]:
# Установка Llama-cpp-python с поддержкой CUDA
# ВАЖНО: Без CMAKE_ARGS она встанет как CPU-only библиотека.
# CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python

In [ ]:
# 1. Скачиваем GGUF файл (около 5 ГБ)
from huggingface_hub import hf_hub_download

print("📥 Скачивание GGUF (если еще нет)...")
model_path = hf_hub_download(
    repo_id="unsloth/Qwen3-8B-GGUF",
    filename="qwen3-8b-instruct-q4_k_m.gguf",
    local_dir="./models"
)
print(f"Файл готов: {model_path}")

In [ ]:
# 2. Запуск через llama-cpp-python
from llama_cpp import Llama

clean_memory()

print("--- Сценарий 1: Full GPU (n_gpu_layers=-1) ---")
# -1 означает "положить все слои в VRAM". Это работает для 7B на 8GB карте.
llm = Llama(
    model_path=model_path,
    n_gpu_layers=-1, 
    n_ctx=4096,
    verbose=False
)

output = llm.create_chat_completion(
    messages=[{"role": "user", "content": "Почему Rust считают безопасным языком?"}],
    max_tokens=100
)
print("Ответ (GPU):", output['choices'][0]['message']['content'])

# Очищаем перед симуляцией гибридного режима
del llm
clean_memory()

print("\n--- Сценарий 2: Hybrid Mode (CPU + GPU) ---")
# Симулируем ситуацию, когда модель не влезает (ставим мало слоев в GPU)
llm_hybrid = Llama(
    model_path=model_path,
    n_gpu_layers=10, # Только 10 слоев на карте, остальные на CPU
    n_ctx=4096,
    verbose=False
)

output = llm_hybrid.create_chat_completion(
    messages=[{"role": "user", "content": "Почему Python такой медленный?"}],
    max_tokens=100
)
print("Ответ (Hybrid):", output['choices'][0]['message']['content'])

del llm_hybrid
clean_memory()

## Ollama + OpenAI API

In [ ]:
import openai

# Перед запуском нужно скачать модель внутри контейнера Ollama
# Нужно выполнить в терминале: docker exec -it rag_ollama ollama pull qwen3:8b

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama" 
)

print("🌐 Запрос к локальной Ollama (Docker)...")

try:
    response = client.chat.completions.create(
        model="qwen3:8b", 
        messages=[
            {"role": "system", "content": "Ты эксперт по DevOps."},
            {"role": "user", "content": "Как мне оптимизировать Docker образ?"},
        ],
        temperature=0.7,
        stream=True
    )
    
    print("Ответ:")
    for chunk in response:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            
except Exception as e:
    print(f"Ошибка: {e}")
    print("💡 Выполнили ли вы 'docker exec -it rag_ollama ollama pull qwen3:8b'?")

## Итоговая таблица

In [ ]:
print("### 🏆 Результаты бенчмарка (RTX 4070 Laptop)")
print(tabulate(results, headers=["Модель", "VRAM (GB)", "Скорость (tok/s)"], tablefmt="github"))